<h2> (1) Import all relevant libraries, layer and classes <h2>


In [ ]:
# General
import os
import time

# Python libraries
import numpy as np, collections
import matplotlib.pyplot as plt
from IPython.display import SVG

# Tensorflow
import tensorflow as tf

# Keras
import keras.backend as k
from keras import optimizers
from keras import losses, regularizers
from keras import initializers
from keras.utils import plot_model, to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.wrappers.scikit_learn import KerasClassifier

# Scikit-learn
from sklearn import metrics
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split, StratifiedKFold, learning_curve, GridSearchCV

# Seaborn
import seaborn as sn

# Pandas
import pandas as pd

# Locale methods and properties
from auxilary_functions import time_stamp
from load_data import load_and_store_data, load_stored_data 
from model import create_compiled_model
from callbacks import get_callbacks 
from plot_pose_sequence import plot_trainings_data, plot_test_data
from plot_confusion_matrix import print_confusion_matrix 
from plot_trainings_history import plot_loss_acc
from plot_data_distribution import plot_data_distribution

<h2> (2) Load and prepare goalkeeper pose data <h2>

In [ ]:
DATA_PATH = "./stored_data/"

# Load data from .npy files 
# (Framrates/Poses-per-sequence: '5' / '10' / '15' / '30' / '45' / '60' / '75' / '90')
X, y = load_stored_data(dir_path=DATA_PATH, num_poses='90')
# X = [samples, timesteps, features]
# y = [samples, labels]

# Split dataset into train and test set (and shuffle them)
X_train, X_test, y_train_temp, y_test_temp = train_test_split(X, y, test_size = 0.1, random_state = 42)

# Get one hot vector from labels
y_train = to_categorical(y_train_temp, num_classes=6)
y_test = to_categorical(y_test_temp, num_classes=6)

# Input features
INPUT = [
    "HEAD_X",
    "HEAD_Y_",
    "BODY_X",
    "BODY_Y",
    "LEFT_ARM_X",
    "LEFT_ARM_Y",
    "RIGHT_ARM_X",
    "RIGHT_ARM_Y",
    "LEFT_LEG_X",
    "LEFT_LEG_Y",
    "RIGHT_LEG_X",
    "RIGHT_LEG_Y",
]

# Output classes
LABELS = [    
    "STAND_BY",
    "LEFT_DIVE",
    "LONG_LEFT_DIVE",
    "RIGHT_DIVE",
    "LONG_RIGHT_DIVE",
    "HIGH_CATCH",
]

# Some debugging info
print("---------------------------------------")
print("---------------------------------------")
print("Bsic information regardin the data:")
print("---------------------------------------")
print("Trainings data...")
print("... input shape=" + str(X_train.shape))
print("... target shape=" + str(y_train.shape))
print("---------------------------------------")
print("Test data...")
print("... input shape=" + str(X_test.shape))
print("... target shape=" + str(y_test.shape))
print("---------------------------------------")
print("---------------------------------------")

In [ ]:
# Plot data distribution (trainings data)
plot_data_distribution(y_train_temp, labels=LABELS)

In [ ]:
# Plot data distribution (teest data)
plot_data_distribution(y_test_temp, labels=LABELS)

<h2> (3) Define constants and additional parameters <h2>

In [ ]:
# Input Data
n_timesteps = X_train.shape[1] #len(X_train[1])  # n-timesteps per series per series
n_features = X_train.shape[2] #len(X_train[0][0])  # n input parameters per timestep

# LSTM Neural Network's internal structure
input_shape = (n_timesteps, n_features)
n_mem_units = 180 # Hidden layer num of features
n_classes = len(LABELS) # n classes (should go up, or should go down)

# Training - Hyperparameter  
learning_rate = 0.0008
optimizers = [optimizers.RMSprop(lr=learning_rate, decay=0.5), 
              optimizers.Adam(lr=learning_rate, decay=0.5),
              optimizers.Nadam(lr=learning_rate)]
num_epochs = 42
batch_size = [256, 512]
dropout = [True, False]

In [ ]:
# Create Parameter Grid 
param_grid = { 
    'optimizer':optimizers, 
    'batch_size':batch_size,
    'dropout':dropout
}

<h2> (4) Define and build sequence model <h2>

In [ ]:
# Weight classes to overcome the unbalanced data
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train_temp), y_train_temp)

# Create model
model = KerasClassifier(build_fn=create_compiled_model, input_shape=input_shape, class_weights)

# Generate Grid Search Model with above parameters
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, verbose=0, refit=True)

<h2> (6) Train model <h2>

In [ ]:
# Reshape model fo CoreML model 
reshaped_X_train = X_train.reshape(X_train.shape[0],-1)
print(reshaped_X_train.shape)

In [ ]:
# Train model on trainings data 
grid_result = grid_search.fit(reshaped_X_train, y_train)

In [ ]:
print("train score - " + str(grid_search.score(reshaped_X_train, y_train)))

reshaped_X_test = X_test.reshape(X_test.shape[0],-1)
print("test score - " + str(grid_search.score(reshaped_X_test, y_test)))

<h2> (7) Predict and evaluate <h2>

In [ ]:
# Show the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
cv_result = pd.DataFrame(grid_search.cv_results_)
cv_result.sort_values('rank_test_score')
cv_result.to_csv(r'results_90p.csv')

In [ ]:
cv_best_param = pd.DataFrame(grid_search.best_params_, index=[0])
cv_best_param.to_csv(r'best_param_90p.csv')

In [ ]:
df = pd.read_csv('best_param_90p.csv') 
var = df['dropout']

In [ ]:
print(var[0])

In [ ]:
best_params = df.to_dict('records')
print(best_params)

In [ ]:
print(best_params[0]['dropout'])